In [1]:
import sys

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


plt.style.reload_library()
plt.style.use(['science'])

# Data Processing Functions

In [2]:
def compute_gaps(df):    
    """ Computes optimality gaps between methods. Assumes the problem are in 
    the minimization form. Hence the extensive form would have a better objective
    as compared to neural network based methods.

    A positive gap would indicate that EF is better than method.

    A negative gap would indicate that EF is worse than method.

    Note: PP is in maximization form. So the interpretation should be reversed.
    
    """
    for index in df.index:

        for test_set in df['downstream'][index].keys():

            ef_obj = df['downstream'][index][test_set]['ef_obj']
            nn_p_obj = df['downstream'][index][test_set]['nn_p_true_obj']
            nn_e_obj = df['downstream'][index][test_set]['nn_e_true_obj']

            if "pp" in index:
                df['downstream'][index][test_set]['ef-nn_p_gap'] = - 100 * (nn_p_obj - ef_obj) / np.abs(ef_obj)
                df['downstream'][index][test_set]['ef-nn_e_gap'] = - 100 * (nn_e_obj - ef_obj) / np.abs(ef_obj)
                df['downstream'][index][test_set]['nn_p-nn_e_gap'] = - 100 * (nn_e_obj - nn_p_obj) / np.abs(nn_p_obj)
            else:
                df['downstream'][index][test_set]['ef-nn_p_gap'] = 100 * (nn_p_obj - ef_obj) / np.abs(ef_obj)
                df['downstream'][index][test_set]['ef-nn_e_gap'] = 100 * (nn_e_obj - ef_obj) / np.abs(ef_obj)
                df['downstream'][index][test_set]['nn_p-nn_e_gap'] = 100 * (nn_e_obj - nn_p_obj) / np.abs(nn_p_obj)

    return df

In [3]:
def get_ef_times_to_best(df, tol=1e-4):    
    """ Gets time that EF takes to reach NN-{P,E} as as well
        as an indicator stating if NN-{P,E} solution quality is better than EF.
    """
    for index in df.index:
        for test_set in df['downstream'][index].keys():

            nn_p_obj = df['downstream'][index][test_set]['nn_p_true_obj']
            nn_e_obj = df['downstream'][index][test_set]['nn_e_true_obj']

            primal_bounds = df.at[index, 'downstream'][test_set]["ef_solving_results"]["primal"]
            times = df.at[index, 'downstream'][test_set]["ef_solving_results"]["time"]

            if "pp" in index:
                nn_p_idx = [n for n,i in enumerate(primal_bounds) if i >= nn_p_obj  - tol]
                nn_e_idx = [n for n,i in enumerate(primal_bounds) if i >= nn_e_obj  - tol]
            else:
                nn_p_idx = [n for n,i in enumerate(primal_bounds) if i <= nn_p_obj  + tol]
                nn_e_idx = [n for n,i in enumerate(primal_bounds) if i <= nn_e_obj  + tol]

            # record time and indicator for NN-P
            if len(nn_p_idx) > 0:
                df['downstream'][index][test_set]['ef_time_to_nn_e'] = times[nn_p_idx[0]]
                df['downstream'][index][test_set]['nn_e_better'] = 0
            else:
                df['downstream'][index][test_set]['ef_time_to_nn_e'] = np.nan
                df['downstream'][index][test_set]['nn_e_better'] = 1

            # record time and indicator for NN-E
            if len(nn_e_idx) > 0:
                df['downstream'][index][test_set]['ef_time_to_nn_p'] = times[nn_e_idx[0]]
                df['downstream'][index][test_set]['nn_p_better'] = 0

            else:
                df['downstream'][index][test_set]['ef_time_to_nn_p'] = np.nan
                df['downstream'][index][test_set]['nn_p_better'] = 1

            # case if NN-P is nan
            if np.isnan(nn_p_obj):
                df['downstream'][index][test_set]['ef_time_to_nn_e'] = 0
                df['downstream'][index][test_set]['nn_e_better'] = 0
            
            # case if NN-E is nan
            if np.isnan(nn_e_obj):
                df['downstream'][index][test_set]['ef_time_to_nn_p'] = 0
                df['downstream'][index][test_set]['nn_p_better'] = 0

    return df

In [4]:
def get_dataframe_for_problem(df_, problem):
    """ Filters columns to be of a specific problem. """
    df = df_.copy()
    rows_to_drop = list(filter(lambda x: problem not in x, df.index))
    df = df.drop(rows_to_drop, axis=0)
    return df

In [5]:
def get_ml_data_df(df_, ml_data_metrics, problem):
    """ Gets data for ML training/data generation. """
    df = df_.copy()
    
    df = get_dataframe_for_problem(df, problem)

    # get only one row per problem
    rows = list(df.index)
    rows_new = list(map(lambda x: "_".join(x.split("_")[:-1]), df.index))
    row_dict = {rows[i]: rows_new[i] for i in range(len(rows_new))}

    df = df.rename(index=row_dict)#.drop_duplicates()
    df = df[ml_data_metrics]
    df = df.drop_duplicates()
    
    df['nn_e_total_time'] = df['nn_e_training_time'] + df['nn_e_data_generation_time']
    df['nn_p_total_time'] = df['nn_p_training_time'] + df['nn_p_data_generation_time']

    return df

In [6]:
def get_agg_df(df, downstream_metrics, problem):
    """ Gets aggregate stats for each scneario setting. """
    agg_df = result_df.copy()
    agg_df[downstream_metrics] = np.nan    
    agg_df = get_dataframe_for_problem(agg_df, problem)

    for index in agg_df.index:
        temp_df = pd.DataFrame(df['downstream'][index]).T
        agg_stat_df = temp_df[downstream_metrics]
        agg_stats = agg_stat_df.mean()

        for i, v in agg_stats.items():
            # sum for nn_better
            if i == "nn_p_better" or i == "nn_e_better":
                agg_df.at[index, i] = agg_stat_df[i].sum()

            # nanmean for time to
            elif i == "ef_time_to_nn_p" or i == "ef_time_to_nn_e":
                ef_time_to = agg_stat_df[i]
                if ef_time_to.isnull().all():
                    agg_df.at[index, i] = np.nan 
                else:
                    agg_df.at[index, i] = ef_time_to.mean()

            # other columns
            else:
                agg_df.at[index, i] = v

    # cast to string with brackets around for displaying in table.
    if "nn_p_better" in agg_df.columns:
        agg_df = agg_df.astype({"nn_p_better" : int})
        agg_df = agg_df.astype({"nn_p_better" : str})
        agg_df["nn_p_better"] = "(" + agg_df["nn_p_better"] + ")"

    if "nn_e_better" in agg_df.columns:
        agg_df = agg_df.astype({"nn_e_better" : int})
        agg_df = agg_df.astype({"nn_e_better" : str})
        agg_df["nn_e_better"] = "(" + agg_df["nn_e_better"] + ")"

    return agg_df[downstream_metrics]

In [7]:
def get_siplib_df(df, downstream_metrics):
    """ Gets SIPLIB SSLP results. """
    rows = df.index
    rows = list(filter(lambda x: 'sslp' in x, rows))
    
    siplib_df = pd.DataFrame(columns=downstream_metrics, index=rows)

    for index in rows:
        siplib_stats = pd.DataFrame(df['downstream'][index]).T[downstream_metrics].loc['siplib']        
        siplib_df.at[index] = siplib_stats

    return siplib_df[downstream_metrics]

In [8]:
def get_combined_df(df, downstream_metrics, problem):
    """ Gets aggregate stats for each scneario setting. """   
    # get rows
    rows = df.index
    rows = list(filter(lambda x: problem in x, rows))

    # get combined rows
    combined_rows = []
    for index in rows:
        for test_set in result_df['downstream'][index].keys():
            combined_rows.append(f"{index}_{test_set}")
           
    # fill df
    combined_df = pd.DataFrame(columns=downstream_metrics, index=combined_rows)
    for index in rows:
        index_df = pd.DataFrame(df['downstream'][index]).T[downstream_metrics]
        for test_set in result_df['downstream'][index].keys():
            combined_df.loc[f"{index}_{test_set}"] = index_df.loc[test_set]
            
    return combined_df

### Load Data

In [10]:
data_dir = '../data/'
data_fp = data_dir + '/results.pkl'

In [11]:
# Read results
with open(data_fp, 'rb') as p:
    results = pickle.load(p)
    
result_df = pd.DataFrame(results).T
print('Shape: ', result_df.shape)

Shape:  (53, 13)


In [12]:
result_df = compute_gaps(result_df)

In [13]:
result_df = get_ef_times_to_best(result_df)

# Main Paper Results

## Table 2: Data Generation and Training Times

In [14]:
def latex_offline_times(df_):
    """ Generate Latex table for training stats.  """

    df = df_.copy()

    df = df.drop(['nn_p_n_samples', 'nn_p_time_per_sample'], axis=1)
    df = df.drop(['nn_e_n_samples', 'nn_e_time_per_sample'], axis=1)
    df = df.drop(['lr_training_time'], axis=1)

    df.rename(columns={
        'nn_p_data_generation_time' : 'NN-P',
        'nn_e_data_generation_time' : 'NN-E',
        'nn_p_training_time' : 'NN-P\ ',
        'nn_e_training_time' : 'NN-E\ ',
        'nn_p_total_time' : 'NN-P\ \ ',
        'nn_e_total_time' : 'NN-E\ \ ',
       }, inplace=True)

    df = df.style.format(thousands=',', precision=2)                

    end_str = '}\n\\caption{Training times.  All times in seconds.}\n'
    end_str += '\\label{tab:tr_times}\n'
    end_str += '\\end{table*}' 

    latex_str = df.to_latex(column_format='l|lll').replace('_','\_').replace('nan', '-').replace('#', '\#')
    latex_str = latex_str.replace('cflp', 'CFLP')
    latex_str = latex_str.replace('sslp', 'SSLP')
    latex_str = latex_str.replace("ip\_c", "INVP").replace("\_i", "\_I").replace("\_b", "\_B")
    latex_str = latex_str.replace("pp", "PP")

    latex_str2 = (latex_str.splitlines())
    latex_str2.insert(1, '\\toprule')
    latex_str2.insert(2, 'Problem & \multicolumn{2}{c}{Data Generation Time} & \multicolumn{2}{c}{Training Time} & \multicolumn{2}{c}{Total Time }')
    latex_str2.insert(3, '\\midrule')
    latex_str2.insert(-1, '\\bottomrule')
    latex_str2.insert(0, '\\begin{table*}[t]\\centering\\resizebox{0.4\\textwidth}{!}{')
    latex_str2[-1] += end_str #'}\end{table*}'#.insert(-1, '\\end{tabular}}')

    print("\n".join(latex_str2))
    print()

In [15]:
ml_data_metrics = [
    'nn_e_n_samples',
    'nn_e_time_per_sample',
    'nn_e_data_generation_time',
    'nn_p_n_samples',
    'nn_p_time_per_sample',
    'nn_p_data_generation_time',
    'nn_e_training_time',
    'nn_p_training_time',
    'lr_training_time',
]

In [16]:
cflp_ml_df = get_ml_data_df(result_df, ml_data_metrics, "cflp")
sslp_ml_df = get_ml_data_df(result_df, ml_data_metrics, "sslp")
ip_ml_df = get_ml_data_df(result_df, ml_data_metrics, "ip")
pp_ml_df = get_ml_data_df(result_df, ml_data_metrics, "pp")

In [17]:
latex_offline_times(cflp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll}
\toprule
Problem & \multicolumn{2}{c}{Data Generation Time} & \multicolumn{2}{c}{Training Time} & \multicolumn{2}{c}{Total Time }
\midrule
 & NN-E & NN-P & NN-E\  & NN-P\  & NN-E\ \  & NN-P\ \  \\
CFLP\_10\_10 & 1,756.55 & 12.12 & 410.10 & 538.42 & 2,166.65 & 550.53 \\
CFLP\_25\_25 & 6,740.48 & 158.61 & 403.82 & 611.60 & 7,144.30 & 770.22 \\
CFLP\_50\_50 & 11,799.16 & 248.00 & 388.89 & 198.80 & 12,188.05 & 446.80 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [18]:
latex_offline_times(sslp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll}
\toprule
Problem & \multicolumn{2}{c}{Data Generation Time} & \multicolumn{2}{c}{Training Time} & \multicolumn{2}{c}{Total Time }
\midrule
 & NN-E & NN-P & NN-E\  & NN-P\  & NN-E\ \  & NN-P\ \  \\
SSLP\_10\_50 & 1,021.84 & 26.61 & 387.55 & 1,019.51 & 1,409.39 & 1,046.12 \\
SSLP\_15\_45 & 1,259.05 & 26.73 & 409.45 & 1,020.20 & 1,668.49 & 1,046.93 \\
SSLP\_5\_25 & 938.86 & 15.98 & 400.34 & 1,025.75 & 1,339.21 & 1,041.73 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [19]:
latex_offline_times(ip_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll}
\toprule
Problem & \multicolumn{2}{c}{Data Generation Time} & \multicolumn{2}{c}{Training Time} & \multicolumn{2}{c}{Total Time }
\midrule
 & NN-E & NN-P & NN-E\  & NN-P\  & NN-E\ \  & NN-P\ \  \\
ip\_B\_E & 13,917.70 & 3.75 & 402.54 & 219.61 & 14,320.24 & 223.35 \\
ip\_B\_H & 13,722.34 & 3.69 & 408.53 & 627.87 & 14,130.88 & 631.56 \\
ip\_I\_E & 15,336.42 & 3.50 & 393.05 & 679.28 & 15,729.47 & 682.78 \\
ip\_I\_H & 12,959.41 & 3.59 & 447.30 & 555.88 & 13,406.72 & 559.47 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [20]:
latex_offline_times(pp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll}
\toprule
Problem & \multicolumn{2}{c}{Data Generation Time} & \multicolumn{2}{c}{Training Time} & \multicolumn{2}{c}{Total Time }
\midrule
 & NN-E & NN-P & NN-E\  & NN-P\  & NN-E\ \  & NN-P\ \  \\
PP & 1,272.40 & 19.12 & 411.90 & 362.46 & 1,684.29 & 381.58 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



## Tables 3-6: Optimziation Results


In [21]:
def latex_overall(df_, report_num_not_found=True):
    """ Generate latex tables for main paper.  """
    df = df_.copy()

    if "cflp" in df.index[0]:
        problem = "CFLP"
    elif "sslp" in df.index[0]:
        problem = "SSLP"
    elif "ip" in df.index[0]:
        problem = "INVP"
    elif "pp" in df.index[0]:
        problem = "PP"
    
    if not report_num_not_found:
        df = df.drop('nn_p_better', axis=1)
        df = df.drop('nn_e_better', axis=1)

    df.rename(columns={
        'ef-nn_e_gap': 'EF-NN-E',
        'ef-nn_p_gap': 'EF-NN-P', 
        'nn_p-nn_e_gap': 'NN-P-NN-E', 
        'ef_time' : 'EF',
        'nn_p_time' : 'NN-P',
        'nn_e_time' : 'NN-E',
        'ef_time_to_nn_p' : 'NN-P\ ',
        'nn_p_better' : '',
        'ef_time_to_nn_e' : 'NN-E\ ',
        'nn_e_better' : '\ ',
       }, inplace=True)

    df = df.style.format(thousands=',', precision=2)\
                  .highlight_min(subset=['EF', 'NN-P', 'NN-E'], props='textbf:--rwrap;', axis=1)#\
                 
    if not report_num_not_found:
        latex_str = df.to_latex(column_format='l|ll|lll|ll')
        n_cols = 8
        n_cols_last_block = 2
    else:
        latex_str = df.to_latex(column_format='l|ll|lll|llll')
        n_cols = 10
        n_cols_last_block = 4
      
    latex_str = latex_str.replace('_','\_').replace('nan', '-').replace('#', '\#')
    latex_str = latex_str.replace('cflp', 'CFLP')
    latex_str = latex_str.replace('sslp', 'SSLP')
    latex_str = latex_str.replace("ip\_c", "INVP").replace("\_i", "\_I").replace("\_b", "\_B")
    latex_str = latex_str.replace("pp", "PP")

    latex_str2 = (latex_str.splitlines())
    latex_str2.insert(1, '\\toprule')
    latex_str2.insert(2, 'Problem & \multicolumn{2}{c}{Gap Difference} & \multicolumn{3}{c}{Solving Time} & \multicolumn{' + str(n_cols_last_block) + '}{c}{EF time to } \\\\')
    latex_str2.insert(3, '\cmidrule{2-' + str(n_cols) + '}')
    latex_str2.insert(5, '\\midrule')
    latex_str2.insert(-1, '\\bottomrule')
    latex_str2.insert(0, '\\begin{table*}[t]\\centering\\resizebox{0.9\\textwidth}{!}{')
    latex_str2[-1] += '}\n\caption{Results for ' + problem + '}\n\label{tab:res_' + problem + '}\n\end{table*}'#.insert(-1, '\\end{tabular}}')

    print("\n".join(latex_str2))
    print()

In [22]:
downstream_metrics = [
    'ef-nn_e_gap',
    'ef-nn_p_gap', 
    'nn_e_time',
    'nn_p_time',
    'ef_time',
    'ef_time_to_nn_e',
    'nn_e_better',
    'ef_time_to_nn_p',
    'nn_p_better',
 ]

In [23]:
cflp_agg_df = get_agg_df(result_df, downstream_metrics, problem='cflp')
sslp_agg_df = get_agg_df(result_df, downstream_metrics, problem='sslp')
ip_agg_df = get_agg_df(result_df, downstream_metrics, problem='ip')
pp_agg_df = get_agg_df(result_df, downstream_metrics, problem='pp')

In [24]:
latex_overall(cflp_agg_df)

\begin{table*}[t]\centering\resizebox{0.9\textwidth}{!}{
\begin{tabular}{l|ll|lll|llll}
\toprule
Problem & \multicolumn{2}{c}{Gap Difference} & \multicolumn{3}{c}{Solving Time} & \multicolumn{4}{c}{EF time to } \\
\cmidrule{2-10}
 & EF-NN-E & EF-NN-P & NN-E & NN-P & EF & NN-E\  & \  & NN-P\  &  \\
\midrule
CFLP\_10\_10\_100 & 1.80 & 0.04 & \textbf{0.76} & 7.64 & 4,371.34 & 114.34 & (0) & 30.16 & (0) \\
CFLP\_10\_10\_500 & 0.00 & 0.32 & \textbf{0.69} & 161.54 & 10,800.07 & 3,002.03 & (0) & 3,104.63 & (0) \\
CFLP\_10\_10\_1000 & -1.32 & -0.85 & \textbf{0.68} & 721.36 & 10,800.46 & 5,876.12 & (7) & 7,235.60 & (8) \\
CFLP\_25\_25\_100 & 4.66 & -0.75 & 28.86 & \textbf{11.33} & 10,800.06 & - & (10) & 109.93 & (0) \\
CFLP\_25\_25\_500 & 1.61 & -3.60 & \textbf{29.30} & 79.30 & 10,800.12 & - & (10) & 950.27 & (1) \\
CFLP\_25\_25\_1000 & 4.32 & -1.11 & \textbf{29.25} & 203.59 & 10,800.44 & - & (10) & 3,909.45 & (0) \\
CFLP\_50\_50\_100 & 1.91 & -1.19 & \textbf{16.09} & 21.68 & 10,800.04 & - & (1

In [25]:
latex_overall(sslp_agg_df)

\begin{table*}[t]\centering\resizebox{0.9\textwidth}{!}{
\begin{tabular}{l|ll|lll|llll}
\toprule
Problem & \multicolumn{2}{c}{Gap Difference} & \multicolumn{3}{c}{Solving Time} & \multicolumn{4}{c}{EF time to } \\
\cmidrule{2-10}
 & EF-NN-E & EF-NN-P & NN-E & NN-P & EF & NN-E\  & \  & NN-P\  &  \\
\midrule
SSLP\_10\_50\_50 & 0.00 & 4.15 & \textbf{0.51} & 15.65 & 10,800.08 & 14.75 & (0) & 205.12 & (0) \\
SSLP\_10\_50\_100 & -0.00 & 4.41 & \textbf{0.51} & 41.56 & 10,800.14 & 95.21 & (0) & 147.45 & (0) \\
SSLP\_10\_50\_500 & -0.14 & 4.16 & \textbf{0.53} & 2,373.73 & 10,802.85 & 2,759.60 & (0) & 7,622.50 & (5) \\
SSLP\_10\_50\_1000 & -11.92 & -7.71 & \textbf{0.55} & 10,857.85 & 10,800.27 & 9,763.43 & (6) & - & (11) \\
SSLP\_10\_50\_2000 & -102.69 & -24.88 & \textbf{0.57} & 11,575.53 & 10,800.34 & 244.34 & (6) & - & (11) \\
SSLP\_15\_45\_5 & 2.27 & 2.28 & \textbf{0.28} & 18.06 & 2.11 & 1.44 & (0) & 0.41 & (0) \\
SSLP\_15\_45\_10 & 1.70 & 1.68 & \textbf{0.28} & 44.47 & 1,977.95 & 331.32 & (0

In [26]:
latex_overall(ip_agg_df, False)

\begin{table*}[t]\centering\resizebox{0.9\textwidth}{!}{
\begin{tabular}{l|ll|lll|ll}
\toprule
Problem & \multicolumn{2}{c}{Gap Difference} & \multicolumn{3}{c}{Solving Time} & \multicolumn{2}{c}{EF time to } \\
\cmidrule{2-8}
 & EF-NN-E & EF-NN-P & NN-E & NN-P & EF & NN-E\  & NN-P\  \\
\midrule
ip\_B\_E\_4 & 11.90 & 4.28 & 0.17 & 0.21 & \textbf{0.01} & 0.01 & 0.01 \\
ip\_B\_E\_9 & 10.49 & 6.43 & 0.16 & 0.41 & \textbf{0.01} & 0.01 & 0.01 \\
ip\_B\_E\_36 & 2.84 & 0.69 & 0.17 & 2.09 & \textbf{0.08} & 0.02 & 0.02 \\
ip\_B\_E\_121 & 3.17 & 4.85 & \textbf{0.18} & 41.40 & 1.97 & 0.02 & 0.02 \\
ip\_B\_E\_441 & 0.51 & 0.99 & \textbf{0.18} & 1,842.97 & 104.31 & 1.00 & 1.09 \\
ip\_B\_E\_1681 & 0.48 & - & \textbf{0.23} & 10,833.81 & 10,800.05 & 0.00 & 18.31 \\
ip\_B\_E\_10000 & -1.92 & - & \textbf{0.33} & 11,156.21 & 10,816.93 & 0.00 & - \\
ip\_B\_H\_4 & 8.81 & 0.96 & 0.21 & 0.34 & \textbf{0.02} & 0.02 & 0.02 \\
ip\_B\_H\_9 & 5.04 & 8.77 & 0.30 & 0.44 & \textbf{0.02} & 0.01 & 0.01 \\
ip\_B\_H\_36

In [27]:
latex_overall(pp_agg_df, False)

\begin{table*}[t]\centering\resizebox{0.9\textwidth}{!}{
\begin{tabular}{l|ll|lll|ll}
\toprule
Problem & \multicolumn{2}{c}{Gap Difference} & \multicolumn{3}{c}{Solving Time} & \multicolumn{2}{c}{EF time to } \\
\cmidrule{2-8}
 & EF-NN-E & EF-NN-P & NN-E & NN-P & EF & NN-E\  & NN-P\  \\
\midrule
PP\_125 & 3.25 & 6.12 & \textbf{1.30} & 150.98 & 10,800.00 & 3.04 & 7,035.18 \\
PP\_216 & 9.06 & 17.55 & \textbf{1.26} & 5,165.09 & 395.08 & 3.77 & 63.41 \\
PP\_343 & 0.67 & 5.78 & \textbf{1.27} & 1,587.81 & 10,800.02 & 44.52 & 1,383.76 \\
PP\_512 & 8.69 & 18.77 & \textbf{1.13} & 10,830.45 & 10,800.00 & 18.15 & 143.80 \\
PP\_729 & 1.38 & 32.00 & \textbf{3.69} & 10,844.09 & 10,800.00 & 54.95 & 6,225.54 \\
PP\_1000 & 5.92 & 325.41 & \textbf{1.38} & 10,862.35 & 10,800.01 & 0.00 & 1,855.05 \\
\bottomrule
\end{tabular}}
\caption{Results for PP}
\label{tab:res_PP}
\end{table*}



# Appendix Results

## Table 8: Data Generation

In [28]:
def latex_appendix_data_times(df_):
    """ Generate Latex table for data generation stats.  """
    df = df_.copy()
    df = df.drop(['nn_p_training_time', 'nn_e_training_time', 'lr_training_time'], axis=1)

    df.rename(columns={
        'nn_p_n_samples' : '# samples',
        'nn_p_time_per_sample' : 'Time per sample',
        'nn_p_data_generation_time' : 'Total time',
        'nn_e_n_samples' : '# samples\ ',
        'nn_e_time_per_sample' : 'Time per sample\ ',
        'nn_e_data_generation_time' : 'Total time\ ',

       }, inplace=True)

    df = df.style.format(thousands=',', precision=2)                

    end_str = '}\n\\caption{Data generation stats and time.  All times in seconds.}\n'
    end_str += '\\label{tab:tr_times}\n'
    end_str += '\\end{table*}' 

    latex_str = df.to_latex(column_format='l|lll|lll').replace('_','\_').replace('nan', '-').replace('#', '\#')
    latex_str = latex_str.replace('cflp', 'CFLP')
    latex_str = latex_str.replace('sslp', 'SSLP')
    latex_str = latex_str.replace("ip\_c", "INVP").replace("\_i", "\_I").replace("\_b", "\_B")
    latex_str = latex_str.replace("pp", "PP")

    latex_str2 = (latex_str.splitlines())
    latex_str2.insert(1, '\\toprule')
    latex_str2.insert(2, 'Problem & \multicolumn{3}{c}{NN-E} & \multicolumn{3}{c}{NN-E}')
    latex_str2.insert(3, '\\midrule')
    latex_str2.insert(-1, '\\bottomrule')
    latex_str2.insert(0, '\\begin{table*}[t]\\centering\\resizebox{0.4\\textwidth}{!}{')
    latex_str2[-1] += end_str #'}\end{table*}'#.insert(-1, '\\end{tabular}}')

    print("\n".join(latex_str2))
    print()

In [29]:
latex_appendix_data_times(cflp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
Problem & \multicolumn{3}{c}{NN-E} & \multicolumn{3}{c}{NN-E}
\midrule
 & \# samples\  & Time per sample\  & Total time\  & \# samples & Time per sample & Total time & nn\_e\_total\_time & nn\_p\_total\_time \\
CFLP\_10\_10 & 5,000 & 0.35 & 1,756.55 & 10,000 & 0.00 & 12.12 & 2,166.65 & 550.53 \\
CFLP\_25\_25 & 5,000 & 1.35 & 6,740.48 & 10,000 & 0.02 & 158.61 & 7,144.30 & 770.22 \\
CFLP\_50\_50 & 5,000 & 2.36 & 11,799.16 & 10,000 & 0.02 & 248.00 & 12,188.05 & 446.80 \\
\bottomrule
\end{tabular}}
\caption{Data generation stats and time.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [30]:
latex_appendix_data_times(sslp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
Problem & \multicolumn{3}{c}{NN-E} & \multicolumn{3}{c}{NN-E}
\midrule
 & \# samples\  & Time per sample\  & Total time\  & \# samples & Time per sample & Total time & nn\_e\_total\_time & nn\_p\_total\_time \\
SSLP\_10\_50 & 5,000 & 0.20 & 1,021.84 & 10,000 & 0.00 & 26.61 & 1,409.39 & 1,046.12 \\
SSLP\_15\_45 & 5,000 & 0.25 & 1,259.05 & 10,000 & 0.00 & 26.73 & 1,668.49 & 1,046.93 \\
SSLP\_5\_25 & 5,000 & 0.19 & 938.86 & 10,000 & 0.00 & 15.98 & 1,339.21 & 1,041.73 \\
\bottomrule
\end{tabular}}
\caption{Data generation stats and time.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [31]:
latex_appendix_data_times(ip_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
Problem & \multicolumn{3}{c}{NN-E} & \multicolumn{3}{c}{NN-E}
\midrule
 & \# samples\  & Time per sample\  & Total time\  & \# samples & Time per sample & Total time & nn\_e\_total\_time & nn\_p\_total\_time \\
ip\_B\_E & 5,000 & 2.78 & 13,917.70 & 10,000 & 0.00 & 3.75 & 14,320.24 & 223.35 \\
ip\_B\_H & 5,000 & 2.74 & 13,722.34 & 10,000 & 0.00 & 3.69 & 14,130.88 & 631.56 \\
ip\_I\_E & 5,000 & 3.07 & 15,336.42 & 10,000 & 0.00 & 3.50 & 15,729.47 & 682.78 \\
ip\_I\_H & 5,000 & 2.59 & 12,959.41 & 10,000 & 0.00 & 3.59 & 13,406.72 & 559.47 \\
\bottomrule
\end{tabular}}
\caption{Data generation stats and time.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [32]:
latex_appendix_data_times(pp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
Problem & \multicolumn{3}{c}{NN-E} & \multicolumn{3}{c}{NN-E}
\midrule
 & \# samples\  & Time per sample\  & Total time\  & \# samples & Time per sample & Total time & nn\_e\_total\_time & nn\_p\_total\_time \\
PP & 5,000 & 0.25 & 1,272.40 & 10,000 & 0.00 & 19.12 & 1,684.29 & 381.58 \\
\bottomrule
\end{tabular}}
\caption{Data generation stats and time.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



## Table 9: Training Times

In [33]:
def latex_appendix_training_times(df_):
    """ Generate Latex table for data generation stats.  """
    df = df_.copy()
    df = df.drop(['nn_p_n_samples', 'nn_p_time_per_sample', 'nn_p_data_generation_time', 'nn_p_total_time'], axis=1)
    df = df.drop(['nn_e_n_samples', 'nn_e_time_per_sample', 'nn_e_data_generation_time', 'nn_e_total_time'], axis=1)

    df.rename(columns={
        'nn_e_training_time' : 'NN-E',
        'nn_p_training_time' : 'NN-P',
        'lr_training_time' : 'LR',
       }, inplace=True)

    df = df.style.format(thousands=',', precision=2)                

    end_str = '}\n\\caption{Training times.  All times in seconds.}\n'
    end_str += '\\label{tab:tr_times}\n'
    end_str += '\\end{table*}' 

    latex_str = df.to_latex(column_format='l|lll|lll').replace('_','\_').replace('nan', '-').replace('#', '\#')
    latex_str = latex_str.replace('cflp', 'CFLP')
    latex_str = latex_str.replace('sslp', 'SSLP')
    latex_str = latex_str.replace("ip\_c", "INVP").replace("\_i", "\_I").replace("\_b", "\_B")
    latex_str = latex_str.replace("pp", "PP")

    latex_str2 = (latex_str.splitlines())
    latex_str2.insert(1, '\\toprule')
    latex_str2.insert(3, '\\midrule')
    latex_str2.insert(-1, '\\bottomrule')
    latex_str2.insert(0, '\\begin{table*}[t]\\centering\\resizebox{0.4\\textwidth}{!}{')
    latex_str2[-1] += end_str #'}\end{table*}'#.insert(-1, '\\end{tabular}}')

    print("\n".join(latex_str2))
    print()

In [34]:
latex_appendix_training_times(cflp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
 & NN-E & NN-P & LR \\
\midrule
CFLP\_10\_10 & 410.10 & 538.42 & 0.02 \\
CFLP\_25\_25 & 403.82 & 611.60 & 0.04 \\
CFLP\_50\_50 & 388.89 & 198.80 & 0.05 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [35]:
latex_appendix_training_times(sslp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
 & NN-E & NN-P & LR \\
\midrule
SSLP\_10\_50 & 387.55 & 1,019.51 & 0.25 \\
SSLP\_15\_45 & 409.45 & 1,020.20 & 0.04 \\
SSLP\_5\_25 & 400.34 & 1,025.75 & 0.02 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [36]:
latex_appendix_training_times(ip_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
 & NN-E & NN-P & LR \\
\midrule
ip\_B\_E & 402.54 & 219.61 & 0.01 \\
ip\_B\_H & 408.53 & 627.87 & 0.02 \\
ip\_I\_E & 393.05 & 679.28 & 0.10 \\
ip\_I\_H & 447.30 & 555.88 & 0.09 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



In [37]:
latex_appendix_training_times(pp_ml_df)

\begin{table*}[t]\centering\resizebox{0.4\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
 & NN-E & NN-P & LR \\
\midrule
PP & 411.90 & 362.46 & 0.02 \\
\bottomrule
\end{tabular}}
\caption{Training times.  All times in seconds.}
\label{tab:tr_times}
\end{table*}



## Table 10-13: Objective Results

In [38]:
def latex_appendix_objective(df):
    """ Generate latex table for detailed objective results. """
    if "cflp" in df.index[0]:
      problem = "CFLP"
    elif "sslp" in df.index[0]:
      problem = "SSLP"
    elif "ip" in df.index[0]:
      problem = "INVP"
    elif "pp" in df.index[0]:
      problem = "PP"

    df.rename(columns={
        'ef_obj' : 'EF',
        'nn_p_true_obj' : 'NN-P',
        'nn_e_true_obj' : 'NN-E', 
        'lr_true_obj' : 'LR',
        'nn_p_pred_obj' : 'NN-P\ ',
        'nn_e_pred_obj' : 'NN-E\ ', 
        'lr_pred_obj' : 'LR\ ',
       }, inplace=True)


    if problem != "PP":
        df = df.style.format(thousands=',', precision=2)\
                    .highlight_min(subset=['EF', 'NN-P', 'NN-E', 'LR'], props='textbf:--rwrap;', axis=1)

    else:
        df = df.style.format(thousands=',', precision=2)\
                    .highlight_max(subset=['EF', 'NN-P', 'NN-E', 'LR'], props='textbf:--rwrap;', axis=1)


    latex_str = df.to_latex(column_format='l|llll|lll')

    latex_str = latex_str.replace('_','\_').replace('nan', '-').replace('#', '\#')
    latex_str = latex_str.replace('cflp', 'CFLP')
    latex_str = latex_str.replace('sslp', 'SSLP')
    latex_str = latex_str.replace("ip\_c", "INVP").replace("\_i", "\_I").replace("\_b", "\_B")
    latex_str = latex_str.replace("pp", "PP")

    latex_str2 = (latex_str.splitlines())
    latex_str2.insert(1, '\\toprule')
    latex_str2.insert(2, 'Problem & \multicolumn{4}{c}{True objective} & \multicolumn{3}{c}{Approximate-MIP objective} \\\\')
    latex_str2.insert(3, '\cmidrule{2-8}')
    latex_str2.insert(5, '\\midrule')
    latex_str2.insert(-1, '\\bottomrule')
    latex_str2.insert(0, '\\begin{table*}[t]\\centering\\resizebox{\\textwidth}{!}{')
    latex_str2[-1] += '}\n\caption{Objective for ' + problem + '}\n\label{tab:obj_' + problem + '}\n\end{table*}'#.insert(-1, '\\end{tabular}}')

    print("\n".join(latex_str2))
    print()

In [39]:
objective_metrics = [
      'nn_e_true_obj', 
      'nn_p_true_obj', 
      'lr_true_obj', 
      'ef_obj',
      'nn_e_pred_obj', 
      'nn_p_pred_obj',      
      'lr_pred_obj', 
 ]

In [40]:
cflp_obj_df = get_agg_df(result_df, objective_metrics, problem='cflp')
sslp_obj_df = get_agg_df(result_df, objective_metrics, problem='sslp')
ip_obj_df = get_agg_df(result_df, objective_metrics, problem='ip')
pp_obj_df = get_agg_df(result_df, objective_metrics, problem='pp')

In [41]:
latex_appendix_objective(cflp_obj_df)

\begin{table*}[t]\centering\resizebox{\textwidth}{!}{
\begin{tabular}{l|llll|lll}
\toprule
Problem & \multicolumn{4}{c}{True objective} & \multicolumn{3}{c}{Approximate-MIP objective} \\
\cmidrule{2-8}
 & NN-E & NN-P & LR & EF & NN-E\  & NN-P\  & LR\  \\
\midrule
CFLP\_10\_10\_100 & 7,121.76 & 6,997.87 & 10,418.87 & \textbf{6,994.77} & 7,079.11 & 6,953.54 & 5,668.09 \\
CFLP\_10\_10\_500 & 7,003.30 & 7,025.32 & 10,410.19 & \textbf{7,003.30} & 7,213.92 & 6,997.96 & 5,679.34 \\
CFLP\_10\_10\_1000 & \textbf{6,994.31} & 7,027.68 & 10,406.08 & 7,088.34 & 7,241.96 & 6,982.23 & 5,658.34 \\
CFLP\_25\_25\_100 & 12,415.58 & \textbf{11,773.01} & 23,833.73 & 11,864.78 & 10,918.02 & 12,024.61 & 10,440.12 \\
CFLP\_25\_25\_500 & 12,359.91 & \textbf{11,726.34} & 23,834.34 & 12,169.12 & 10,918.02 & 11,976.81 & 10,387.23 \\
CFLP\_25\_25\_1000 & 12,352.73 & \textbf{11,709.90} & 23,833.85 & 11,842.37 & 10,918.02 & 11,970.40 & 10,377.68 \\
CFLP\_50\_50\_100 & 25,824.64 & \textbf{25,036.99} & 45,465.37 & 25,

In [42]:
latex_appendix_objective(cflp_obj_df)

\begin{table*}[t]\centering\resizebox{\textwidth}{!}{
\begin{tabular}{l|llll|lll}
\toprule
Problem & \multicolumn{4}{c}{True objective} & \multicolumn{3}{c}{Approximate-MIP objective} \\
\cmidrule{2-8}
 & NN-E & NN-P & LR & EF & NN-E\  & NN-P\  & LR\  \\
\midrule
CFLP\_10\_10\_100 & 7,121.76 & 6,997.87 & 10,418.87 & \textbf{6,994.77} & 7,079.11 & 6,953.54 & 5,668.09 \\
CFLP\_10\_10\_500 & 7,003.30 & 7,025.32 & 10,410.19 & \textbf{7,003.30} & 7,213.92 & 6,997.96 & 5,679.34 \\
CFLP\_10\_10\_1000 & \textbf{6,994.31} & 7,027.68 & 10,406.08 & 7,088.34 & 7,241.96 & 6,982.23 & 5,658.34 \\
CFLP\_25\_25\_100 & 12,415.58 & \textbf{11,773.01} & 23,833.73 & 11,864.78 & 10,918.02 & 12,024.61 & 10,440.12 \\
CFLP\_25\_25\_500 & 12,359.91 & \textbf{11,726.34} & 23,834.34 & 12,169.12 & 10,918.02 & 11,976.81 & 10,387.23 \\
CFLP\_25\_25\_1000 & 12,352.73 & \textbf{11,709.90} & 23,833.85 & 11,842.37 & 10,918.02 & 11,970.40 & 10,377.68 \\
CFLP\_50\_50\_100 & 25,824.64 & \textbf{25,036.99} & 45,465.37 & 25,

In [43]:
latex_appendix_objective(sslp_obj_df)

\begin{table*}[t]\centering\resizebox{\textwidth}{!}{
\begin{tabular}{l|llll|lll}
\toprule
Problem & \multicolumn{4}{c}{True objective} & \multicolumn{3}{c}{Approximate-MIP objective} \\
\cmidrule{2-8}
 & NN-E & NN-P & LR & EF & NN-E\  & NN-P\  & LR\  \\
\midrule
SSLP\_10\_50\_50 & -354.96 & -340.28 & -63.00 & \textbf{-354.96} & -346.42 & -366.59 & -302.35 \\
SSLP\_10\_50\_100 & \textbf{-345.86} & -330.61 & -49.62 & -345.86 & -346.42 & -355.68 & -292.58 \\
SSLP\_10\_50\_500 & \textbf{-349.54} & -334.53 & -54.68 & -349.05 & -346.42 & -361.36 & -296.30 \\
SSLP\_10\_50\_1000 & \textbf{-350.07} & -336.48 & -55.45 & -318.98 & -346.42 & -347.86 & -297.01 \\
SSLP\_10\_50\_2000 & \textbf{-350.07} & -215.75 & -54.72 & -172.73 & -346.42 & -212.44 & -296.38 \\
SSLP\_15\_45\_5 & -249.51 & -249.58 & 8,650.22 & \textbf{-255.55} & -249.58 & -236.83 & -45.44 \\
SSLP\_15\_45\_10 & -252.89 & -252.95 & 8,595.76 & \textbf{-257.41} & -249.58 & -243.05 & -48.59 \\
SSLP\_15\_45\_15 & -254.58 & -254.16 & 8,68

In [44]:
latex_appendix_objective(ip_obj_df)

\begin{table*}[t]\centering\resizebox{\textwidth}{!}{
\begin{tabular}{l|llll|lll}
\toprule
Problem & \multicolumn{4}{c}{True objective} & \multicolumn{3}{c}{Approximate-MIP objective} \\
\cmidrule{2-8}
 & NN-E & NN-P & LR & EF & NN-E\  & NN-P\  & LR\  \\
\midrule
ip\_B\_E\_4 & -50.22 & -54.56 & -46.25 & \textbf{-57.00} & -59.35 & -50.56 & -64.22 \\
ip\_B\_E\_9 & -53.11 & -55.52 & -53.11 & \textbf{-59.33} & -58.95 & -55.00 & -64.22 \\
ip\_B\_E\_36 & -59.48 & -60.80 & -58.86 & \textbf{-61.22} & -59.47 & -57.93 & -64.22 \\
ip\_B\_E\_121 & -60.32 & -59.27 & -61.06 & \textbf{-62.29} & -59.61 & -59.06 & -64.22 \\
ip\_B\_E\_441 & -61.00 & -60.71 & -59.91 & \textbf{-61.32} & -59.73 & -59.66 & -64.22 \\
ip\_B\_E\_1681 & -60.34 & - & -59.30 & \textbf{-60.63} & -59.88 & inf & -64.22 \\
ip\_B\_E\_10000 & \textbf{-60.11} & - & -58.68 & -58.98 & -59.97 & inf & -64.22 \\
ip\_B\_H\_4 & -51.75 & -56.20 & -51.75 & \textbf{-56.75} & -59.70 & -51.80 & -61.29 \\
ip\_B\_H\_9 & -56.56 & -54.33 & -56.56 & \te

In [45]:
latex_appendix_objective(pp_obj_df)

\begin{table*}[t]\centering\resizebox{\textwidth}{!}{
\begin{tabular}{l|llll|lll}
\toprule
Problem & \multicolumn{4}{c}{True objective} & \multicolumn{3}{c}{Approximate-MIP objective} \\
\cmidrule{2-8}
 & NN-E & NN-P & LR & EF & NN-E\  & NN-P\  & LR\  \\
\midrule
PP\_125 & 264.30 & 256.48 & -107.18 & \textbf{273.19} & 195.12 & 190.54 & 68.37 \\
PP\_216 & 200.29 & 181.59 & 46.56 & \textbf{220.25} & 195.12 & 167.82 & 68.37 \\
PP\_343 & 206.38 & 195.76 & -26.07 & \textbf{207.77} & 195.12 & 180.48 & 68.37 \\
PP\_512 & 204.41 & 181.83 & -65.15 & \textbf{223.86} & 195.12 & 171.73 & 68.37 \\
PP\_729 & 219.42 & 151.28 & 40.91 & \textbf{222.48} & 195.12 & 77.74 & 68.37 \\
PP\_1000 & 202.50 & -485.19 & -9.09 & \textbf{215.25} & 195.12 & -449.07 & 68.37 \\
\bottomrule
\end{tabular}}
\caption{Objective for PP}
\label{tab:obj_PP}
\end{table*}



## Table 14: SSLP SIPLIB Results

In [46]:
def compute_siplib_gaps(df, siplib_opt_sol_dict):    
    """  Computes optimality gaps from SIPLIB SSLP instances. """
    for index in df.index:
        if 'sslp' not in index:
            continue

        for test_set in df['downstream'][index].keys():
            if test_set != "siplib":
                continue

            ef_obj = df['downstream'][index][test_set]['ef_obj']
            nn_p_obj = df['downstream'][index][test_set]['nn_p_true_obj']
            nn_e_obj = df['downstream'][index][test_set]['nn_e_true_obj']

            df['downstream'][index][test_set]['ef_gap'] = 100 * (ef_obj - siplib_opt_sol_dict[index]) / np.abs(siplib_opt_sol_dict[index])
            df['downstream'][index][test_set]['nn_p_gap'] = 100 * (nn_p_obj - siplib_opt_sol_dict[index]) / np.abs(siplib_opt_sol_dict[index])
            df['downstream'][index][test_set]['nn_e_gap'] = 100 * (nn_e_obj - siplib_opt_sol_dict[index]) / np.abs(siplib_opt_sol_dict[index])

            if df['downstream'][index][test_set]['ef_gap'] < 1e-4:
                df['downstream'][index][test_set]['ef_gap'] = 0.0
            if df['downstream'][index][test_set]['nn_p_gap'] < 1e-4:
                df['downstream'][index][test_set]['nn_p_gap'] = 0.0
            if df['downstream'][index][test_set]['nn_e_gap'] < 1e-4:
                df['downstream'][index][test_set]['nn_e_gap'] = 0.0
                
    return df

In [47]:
def latex_sslp_siplib(df):
    """ Generate latex table for SIPLIB SSLP instances. """
    df.rename(columns={
          'ef_gap' : 'EF',
          'nn_p_gap' : "NN-P",
          'nn_e_gap' : "NN-E",
          'ef_time' : "EF\ ",
          'nn_p_time' : "NN-P\ ",
          'nn_e_time' : "NN-E\ ",
       }, inplace=True)
    
    df = df.style.format(thousands=',', precision=2)\
                 .highlight_min(subset=['EF', 'NN-P', 'NN-E'], props='textbf:--rwrap;', axis=1)\
                 .highlight_min(subset=['EF\ ', 'NN-P\ ', 'NN-E\ '], props='textbf:--rwrap;', axis=1)#

    siplib_caption = "SSLP SIPLib gap and time compairison among methods. Optimal SIPLib instances values obtained from \\cite{ahmed2015siplib}.  All times in seconds."

    latex_str = df.to_latex(column_format='l|lll|lll')

    latex_str = latex_str.replace('_','\_').replace('nan', '-').replace('#', '\#')
    latex_str = latex_str.replace('cflp', 'CFLP')
    latex_str = latex_str.replace('sslp', 'SSLP')
    latex_str = latex_str.replace("ip\_c", "INVP").replace("\_i", "\_I").replace("\_b", "\_B")
    latex_str = latex_str.replace("pp", "PP")

    latex_str2 = (latex_str.splitlines())
    latex_str2.insert(1, '\\toprule')
    latex_str2.insert(2, 'Problem & \multicolumn{3}{c}{Gap to Optimal} & \multicolumn{3}{c}{Solving Time} \\\\')
    latex_str2.insert(3, '\cmidrule{2-7}')
    latex_str2.insert(5, '\\midrule')
    latex_str2.insert(-1, '\\bottomrule')
    latex_str2.insert(0, '\\begin{table*}[t]\\centering\\resizebox{\\textwidth}{!}{')
    latex_str2[-1] += '}\n\caption{' + siplib_caption + '}\n\label{tab:res_siplib}\n\end{table*}'#.insert(-1, '\\end{tabular}}')

    print("\n".join(latex_str2))
    print()


In [48]:
siplib_opt_sol_dict = {
    'sslp_5_25_50' : -121.600, 
    'sslp_5_25_100' : -127.370,
    'sslp_10_50_50' : -364.640, 
    'sslp_10_50_100' : -354.190, 
    'sslp_10_50_500' : -349.136, 
    'sslp_10_50_1000' : -351.711, 
    'sslp_10_50_2000' : -347.262, 
    'sslp_15_45_5' : -262.400, 
    'sslp_15_45_10' : -260.500, 
    'sslp_15_45_15' : -253.602, 
}

siplib_metrics = [
    'nn_e_gap',
    'nn_p_gap',
    'ef_gap',
    'nn_e_time',
    'nn_p_time',
    'ef_time',
 ]

In [49]:
result_df = compute_siplib_gaps(result_df, siplib_opt_sol_dict)

In [50]:
sslp_siplib_df = get_siplib_df(result_df, siplib_metrics)

In [51]:
latex_sslp_siplib(sslp_siplib_df)

\begin{table*}[t]\centering\resizebox{\textwidth}{!}{
\begin{tabular}{l|lll|lll}
\toprule
Problem & \multicolumn{3}{c}{Gap to Optimal} & \multicolumn{3}{c}{Solving Time} \\
\cmidrule{2-7}
 & NN-E & NN-P & EF & NN-E\  & NN-P\  & EF\  \\
\midrule
SSLP\_10\_50\_50 & \textbf{0.00} & 4.05 & \textbf{0.00} & \textbf{0.49} & 19.30 & 10,800.02 \\
SSLP\_10\_50\_100 & \textbf{0.00} & 4.24 & \textbf{0.00} & \textbf{0.51} & 34.43 & 10,800.04 \\
SSLP\_10\_50\_500 & \textbf{0.00} & 4.19 & 0.00 & \textbf{0.58} & 799.69 & 10,800.14 \\
SSLP\_10\_50\_1000 & \textbf{0.00} & \textbf{0.00} & 28.64 & \textbf{0.55} & 10,857.87 & 10,800.12 \\
SSLP\_10\_50\_2000 & \textbf{0.00} & 10.49 & 51.24 & \textbf{0.67} & 14,436.75 & 10,800.21 \\
SSLP\_15\_45\_5 & 1.22 & \textbf{0.00} & \textbf{0.00} & \textbf{0.26} & 25.24 & 3.62 \\
SSLP\_15\_45\_10 & 0.46 & 0.58 & \textbf{0.00} & \textbf{0.26} & 49.06 & 3.67 \\
SSLP\_15\_45\_15 & 0.79 & 0.16 & \textbf{0.00} & \textbf{0.25} & 76.58 & 4.80 \\
SSLP\_5\_25\_50 & \textbf{0.0